# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv('../WeatherPy/City_Weather.csv')
city_weather_df.dropna(inplace=True)
city_weather_df


,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
1,Bethel,41.3712,-73.4140,30.99,93,90,5.75,US,1613445823
2,Saint-François,46.4154,3.9054,37.00,87,0,2.30,FR,1613445823
3,Ushuaia,-54.8000,-68.3000,55.40,58,75,2.30,AR,1613445662
4,Mar del Plata,-38.0023,-57.5575,69.01,100,90,5.75,AR,1613445721
5,Kruisfontein,-34.0033,24.7314,59.00,95,64,1.99,ZA,1613445825
...,...,...,...,...,...,...,...,...,...
539,Vredendal,-31.6683,18.5012,58.14,60,0,9.26,ZA,1613446374
540,Kushima,31.4583,131.2333,55.40,46,20,18.41,JP,1613446375
541,Eirunepé,-6.6603,-69.8736,74.10,96,100,2.37,BR,1613446376
542,Tocopilla,-22.0920,-70.1979,67.50,67,0,4.18,CL,1613446224


In [3]:
city_weather_df.dtypes

City                object
City Latitude      float64
City Longitude     float64
Max Temperature    float64
Humidity             int64
Cloud Coverage       int64
Wind Speed         float64
City Country        object
City Datetime        int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = city_weather_df[["City Latitude", "City Longitude"]]

# Fill NaN values and convert humidity to float
humidity = city_weather_df["Humidity"].astype(float)
humidity

1       93.0
2       87.0
3       58.0
4      100.0
5       95.0
       ...  
539     60.0
540     46.0
541     96.0
542     67.0
543     80.0
Name: Humidity, Length: 540, dtype: float64

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
# Ideal weather conditions
# Temperature:  75 - 80 degrees F
# Humidity:  45 - 55%
# Wind Speed: 4 - 12mph
# Cloud Coverage: 0 - 20%

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Narrow down the cities to fit weather conditions
hotel_df = city_weather_df.loc[city_weather_df["Humidity"] >= 40]
hotel_df = hotel_df.loc[city_weather_df["Humidity"] <= 55]
hotel_df = hotel_df.loc[city_weather_df["Max Temperature"] >= 65]
hotel_df = hotel_df.loc[city_weather_df["Max Temperature"] <= 85]
hotel_df = hotel_df.loc[city_weather_df["Wind Speed"] >= 0]
hotel_df = hotel_df.loc[city_weather_df["Wind Speed"] <= 15]
hotel_df = hotel_df.loc[city_weather_df["Cloud Coverage"] >= 0]
hotel_df = hotel_df.loc[city_weather_df["Cloud Coverage"] <= 50]

hotel_df["Hotel Name"] = ""
hotel_df

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime,Hotel Name
45,New Norfolk,-42.7826,147.0587,80.01,41,37,3.00,AU,1613446045,
164,Los Llanos de Aridane,28.6585,-17.9182,66.00,45,40,2.30,ES,1613446122,
277,Itoman,26.1247,127.6694,66.00,55,20,8.05,JP,1613446195,
283,Kosum Phisai,16.2486,103.0674,82.99,47,32,3.44,TH,1613446199,
341,Port Lincoln,-34.7333,135.8667,79.88,41,35,10.38,AU,1613446101,
454,Kyaukse,21.6000,96.1333,68.00,45,0,2.30,MM,1613446320,
458,Talen,23.5667,76.7167,72.25,40,31,6.46,IN,1613446322,
470,Hirara,24.8000,125.2833,71.60,53,20,13.80,JP,1613446330,
476,Simao,22.7886,100.9748,70.07,40,0,2.35,CN,1613446109,


In [13]:
hotel_df.dtypes
hotel_df["City Latitude"].astype(object)
hotel_df["City Longitude"].astype(object)

45     147.059
164   -17.9182
277    127.669
283    103.067
341    135.867
454    96.1333
458    76.7167
470    125.283
476    100.975
Name: City Longitude, dtype: object

In [20]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "http://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    name_address

                      City  City Latitude  City Longitude  Max Temperature  \
45             New Norfolk       -42.7826        147.0587            80.01   
164  Los Llanos de Aridane        28.6585        -17.9182            66.00   
277                 Itoman        26.1247        127.6694            66.00   
283           Kosum Phisai        16.2486        103.0674            82.99   
341           Port Lincoln       -34.7333        135.8667            79.88   
454                Kyaukse        21.6000         96.1333            68.00   
458                  Talen        23.5667         76.7167            72.25   
470                 Hirara        24.8000        125.2833            71.60   
476                  Simao        22.7886        100.9748            70.07   

     Humidity  Cloud Coverage  Wind Speed City Country  City Datetime  \
45         41              37        3.00           AU     1613446045   
164        45              40        2.30           ES     1613446122   


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [21]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["City Latitude", "City Longitude"]]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                            info_box_template=[f"Hotel Name: {name}" for name in hotel_info])
fig.add_layer(markers)

# Display figure
fig

TypeError: marker_layer() got an unexpected keyword argument 'info_box_template'